In [2]:
import numpy as np
import csv
import string
import itertools
import re
from nltk import WordNetLemmatizer, word_tokenize,sent_tokenize,  RegexpTokenizer
from nltk.tokenize.casual import TweetTokenizer
from nltk import FreqDist
lemmatize = WordNetLemmatizer()
unknown_token = "UNKNOWN_TOKEN"
pad_token = "PAD_TOKEN"
tweetknzr = TweetTokenizer()
max_sentance_length = 100

def clean_text(text):
    tokens = word_tokenize(text)
    clean_tokens = [lemmatize.lemmatize(token.lower().strip()) for token in tokens]
    return ' '.join(clean_tokens)
def clean_sentance(text):
    sentences = sent_tokenize(data.lower())
    sentences = [tweetknzr.tokenize(sent) for sent in sentences]
    return sentences

def pad_sentances(sentance):
    padded = []
    sentance = word_tokenize(sentance)
    
    pad_len = max_sentance_length - len(sentance)
    if pad_len > 0:
        for _ in range(pad_len):
            padded.append(pad_token)
        
    for word in sentance:
        padded.append(word)   
    return padded

def flatten_comments(comments):
    flattened = []
    for comment in comments:
        for sentance in sent_tokenize(comment):
            flattened.append(sentance)
    return flattened

def truncate_sentances(sentance):
    if len(sentance) > max_sentance_length:
        sentance = sentance[:max_sentance_length]
    return sentance

## make main in a second
with open("output_comments_query_crypto_all.txt") as f:
    data = f.read()
    data = data.replace('\n','')


raw_sentances = clean_sentance(data)


In [35]:
def make_less_spammy(raw_sentances):
    allowed = []
    spam_chars = ['/','(',')',':',';']
    for sentance in raw_sentances:
        tmp = []
        for char in ' '.join(sentance):
            if char not in spam_chars:
                tmp.append(char)

        text = ''.join(tmp)
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r' ,', ',', text)
        text = re.sub(r'-', ' ', text)
        text = re.sub(r'\(', '', text)
        text = re.sub(r'\[', '', text)
        text = re.sub(r' r ', '', text) 
        text = re.sub(r" ’ ", "’", text) 
        text = re.sub(r'$ ', '$', text) 
        text = re.sub(r' k ', 'k', text)
        text = re.sub(r'>', '', text)
        text = re.sub(r'\$ ', '$', text)
        text = re.sub(r' % ', '% ', text)
        text = re.sub(r'\s+([?.!"])', r'\1', text) 
        text = re.sub(']  ','',text)
        text = re.sub(r'_','',text)
        text = re.sub(r'^','',text)
        text = re.sub(r'\.(?! )', '. ', re.sub(r' +', ' ', text))
        text = re.sub('\.\.\.\.?', '', text)  # remove ellipses)
        text = re.sub(r'[^ A-Za-z0-9.,%$!?]', '', text)  # other punctuation
        tmp = [x for x in text.split(' ') if x]
        text = ' '.join(tmp)
        allowed.append(text)
    return allowed

In [36]:
allowed = make_less_spammy(raw_sentances)
flattend = flatten_comments(allowed)

truncated = [truncate_sentances(sent) for sent in flattend]
lengths = [len(sent) for sent in truncated]
max(lengths)

100

In [37]:
%time
padded = [pad_sentances(sent) for sent in truncated]
word_tokenized_sentences = padded

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [38]:
lengths = [len(sent) for sent in word_tokenized_sentences]
min(lengths)

100

In [39]:
vocab_size = 25000
word_freq = FreqDist(itertools.chain(*word_tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))
vocab = word_freq.most_common(len(word_tokenized_sentences)-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])


# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(word_tokenized_sentences):
    word_tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    

Found 28668 unique words tokens.


In [40]:
import gensim

model = gensim.models.Word2Vec(word_tokenized_sentences, size=100, window=5, min_count=1, workers=2,sg=1)
model.train(word_tokenized_sentences, total_examples=len(word_tokenized_sentences),epochs=10)


(8898420, 65601000)

In [41]:
model.wv.most_similar('vitalik' ,topn=10) #creator of ethereum 

/Users/mcassettix/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('misquoting', 0.7109144330024719),
 ('liars', 0.7064816951751709),
 ('buterin', 0.638586699962616),
 ('csw', 0.62827068567276),
 ('taylor', 0.6084359288215637),
 ('bcash', 0.6077585220336914),
 ('vlad', 0.6024558544158936),
 ('consensys', 0.5981085896492004),
 ('cannon', 0.5963471531867981),
 ('mlm', 0.5903977751731873)]

In [42]:
model.save('mymodel_tagged') ## this isn't used for exploratory analysis, it's used for tagging
import pickle
with open('cleaned_sentances_tagged_padded.pkl','wb') as fp:
    pickle.dump(word_tokenized_sentences,fp)